# IFastAPI

> Running [fastapi](https://github.com/tiangolo/fastapi) under [uvicorn](https://github.com/encode/uvicorn) inside [IPython](https://github.com/ipython/ipython)

In [2]:
import asyncio, uvicorn, fastapi, pydantic, IPython, ujson, tornado.ioloop

## Make an App

In [3]:
app = fastapi.FastAPI(title="IFastAPI")

## Make a route

In [4]:
@app.get("/")
def read_root():
    return {"Hello": "World"}

## Serve in-loop with `uvicorn`

In [5]:
config = uvicorn.Config(app)
server = uvicorn.Server(config=config)

The actual serving will live in an `asyncio.Task`

In [6]:
if __name__ == "__main__":
    task = asyncio.ensure_future(server.serve())

INFO:     Started server process [18509]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


## Live: Modelling the domain

In [8]:
class Item(pydantic.BaseModel):
    """
    a lovely model
    """
    name: str
    price: float
    is_offer: bool = None
IPython.display.JSON(ujson.loads(Item.schema_json()))

<IPython.core.display.JSON object>

In [9]:
DB = {}

## Live: using the domain

In [10]:
@app.get("/items/{item_id}", response_model=Item)
def read_item(item_id: int, q: str = None) -> Item:
    return DB[item_id]

@app.put("/items/{item_id}")
def create_item(item_id: int, item: Item):
    DB[item_id] = item
    return {"item_name": item.name, "item_id": item_id}

# openapi gets cached, force reloading it
app.openapi_schema = None

## All done

In [12]:
if __name__ == "__main__":
    tornado.ioloop.IOLoop.current().add_callback(server.shutdown)

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
